# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch==1.4.0
import torch
print(torch.__version__)

     |████████████████████████████████| 753.4 MB 6.1 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
1.4.0


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning) 

In [ ]:
!pip install stanfordnlp
import stanfordnlp
stanfordnlp.download('en')

     |████████████████████████████████| 158 kB 9.7 MB/s 
Using the default treebank "en_ewt" for language "en".
Would you like to download the models for: en_ewt now? (Y/n)

Default download directory: /root/stanfordnlp_resources
Hit enter to continue or type an alternate directory.

Download location: /root/stanfordnlp_resources/en_ewt_models.zip


100%|██████████| 235M/235M [00:40<00:00, 5.87MB/s]



Download complete.  Models saved to: /root/stanfordnlp_resources/en_ewt_models.zip
Extracting models file for: en_ewt
Cleaning up...Done.


In [ ]:
from xml.etree import cElementTree as ET
import pandas as pd
import string
import spacy
import re
import random
import sklearn
from sklearn.model_selection import train_test_split
import csv
import numpy as np

In [ ]:
root_dir = "/content/drive/MyDrive/NLP Project/Topic 5/"
train_folder = "Train/SemEval'14-ABSA-TrainData_v2 & AnnotationGuidelines/"
laptop_train_file = "Laptop_Train_v2.xml"
restaurant_train_file = "Restaurants_Train_v2.xml"
test_1_folder = "Test1/ABSA_TestData_PhaseA/ABSA_TestData_PhaseA/"
laptop_test_1_file = "Laptops_Test_Data_PhaseA.xml"
restaurant_test_1_file = "Restaurants_Test_Data_PhaseA.xml"
test_2_folder = "Test2/ABSA_TestData_PhaseB/"
laptop_test_2_file = "Laptops_Test_Data_phaseB.xml"
restaurant_test_2_file = "Restaurants_Test_Data_phaseB.xml"

# Data Extraction & Cleaning

In [ ]:
class AspectWord():
    def __init__(self, aspect_term, sentiment_terms, polarity = None):
        # Words
        self.aspect_term = aspect_term
        # List of Words
        self.sentiment_terms = sentiment_terms
        # Polarity: +,-,neutral
        self.polarity = polarity

    def __str__(self):
        return f"Aspect Term: {self.aspect_term} Sentiment Terms: {self.sentiment_terms} Polarity: {self.polarity}"
    
    def __repr__(self):
        return self.__str__()

class AspectCategory():
    def __init__(self, aspect_category, polarity = None):
        # Words
        self.aspect_category = aspect_category
        # Polarity: +,-,neutral
        self.polarity = polarity

    def __str__(self):
        return f"Aspect Category: {self.aspect_category} Polarity: {self.polarity}"
    
    def __repr__(self):
        return self.__str__()

In [ ]:
class Sentence():
    def __init__(self, sentence_id, sentence, data_type = "Train",
                 actual_sentence_id = None, actual_aspect_words = [], actual_aspect_categories = []): 
        # Int
        self.sentence_id = sentence_id
        # String
        self.sentence = sentence
        # String
        self.data_type = data_type
        # String
        self.actual_sentence_id = actual_sentence_id
        # List of AspectWord
        self.actual_aspect_words = actual_aspect_words
        # List of AspectWord
        self.generated_aspect_words = []
        # List of AspectCategory
        self.actual_aspect_categories = actual_aspect_categories
        # Polarity
        self.actual_sentence_polarity = None
        # Polarity
        self.generated_sentence_polarity = None

    def calculate_actual_polarity(self):
        temp_polarity = 0
        for actual_aspect_word in self.actual_aspect_words:
          if(actual_aspect_word.polarity == "positive"):
            temp_polarity+=1
          elif(actual_aspect_word.polarity == "negative"):
            temp_polarity-=1
        if(temp_polarity > 0):
          self.actual_sentence_polarity = "positive"
        elif(temp_polarity < 0):
          self.actual_sentence_polarity = "negative"
        else:
          self.actual_sentence_polarity = "neutral"

    def calculate_generated_polarity(self):
        temp_polarity = 0
        for generated_aspect_word in self.generated_aspect_words:
          if(generated_aspect_word.polarity == "positive"):
            temp_polarity+=1
          elif(generated_aspect_word.polarity == "negative"):
            temp_polarity-=1
        if(temp_polarity > 0):
          self.generated_sentence_polarity = "positive"
        elif(temp_polarity < 0):
          self.generated_sentence_polarity = "negative"
        else:
          self.generated_sentence_polarity = "neutral"

    def __str__(self):
        if self.data_type == "Train":
            return f"ID: {self.sentence_id} Sentence: {self.sentence} {self.actual_aspect_words}"
        else:
            return f"ID: {self.sentence_id} Sentence: {self.sentence} {self.generated_aspect_words}"
    
    def __repr__(self):
        return self.__str__()

In [ ]:
def xml_to_sentences(path, data_type = "Train"):
    data = []
    tree = ET.parse(path)
    root = tree.getroot()
    id = 1
    for page in root.findall('sentence'):
        sentence_id = id
        sentence = page[0].text
        actual_sentence_id = page.attrib["id"]
        actual_aspect_terms = []
        actual_polarity = []
        if (data_type == "Train"):
            if len(page)> 1 and page[1].tag == "aspectTerms":
                aspect_terms_data = [x.attrib for x in page[1]]
                aspect_categories_data = None
                
                # Ignore Sentences without aspect terms if training data
                aspect_words = []
                for x in aspect_terms_data:
                    aspect_words.append(AspectWord(x['term'],[],x['polarity']))

                aspect_categories = []
                if len(page) > 2:
                    aspect_categories_data = [x.attrib for x in page[2]]
                    for x in aspect_categories_data:
                        aspect_categories.append(AspectWord(x['category'],x['polarity']))
                id += 1
                curr_sentence = Sentence(sentence_id, sentence, data_type, actual_sentence_id, aspect_words, aspect_categories)
                curr_sentence.calculate_actual_polarity()
                data.append(curr_sentence)
        elif (data_type == "Test-2"):
            if len(page)> 1 and page[1].tag == "aspectTerms":
                aspect_terms_data = [x.attrib for x in page[1]]
                aspect_categories_data = None
                
                # Ignore Sentences without aspect terms if training data
                aspect_words = []
                for x in aspect_terms_data:
                    aspect_words.append(AspectWord(x['term'],[]))

                aspect_categories = []
                if len(page) > 2:
                    aspect_categories_data = [x.attrib for x in page[2]]
                    for x in aspect_categories_data:
                        aspect_categories.append(AspectWord(x['category'],[]))
                id += 1
                curr_sentence = Sentence(sentence_id, sentence, data_type, actual_sentence_id, aspect_words, aspect_categories)
                curr_sentence.calculate_actual_polarity()
                data.append(curr_sentence)
        elif (data_type == "Test"):
            curr_sentence = Sentence(sentence_id, sentence, data_type, actual_sentence_id)
            data.append(curr_sentence)
            id += 1
        else:
            print("Incorrect Data Type")
            return None
    return data

In [ ]:
def data_list_to_df(data_list, aspect_terms = "actual", aspect_polarity = True, no_of_rows = 5):
    
    data = pd.DataFrame(columns = ["ID","Sentence","Actual Aspect Terms","Actual Sentiment Terms","Actual Polarities","Generated Aspect Terms","Generated Sentiment Terms","Generated Polarities"])
    
    for i in range(min(len(data_list), no_of_rows)):
        data.loc[len(data.index)] = [data_list[i].sentence_id, data_list[i].sentence, [x.aspect_term for x in data_list[i].actual_aspect_words], [x.sentiment_terms for x in data_list[i].actual_aspect_words], [x.polarity for x in data_list[i].actual_aspect_words], [x.aspect_term for x in data_list[i].generated_aspect_words], [x.sentiment_terms for x in data_list[i].generated_aspect_words], [x.polarity for x in data_list[i].generated_aspect_words]] 
    
    if aspect_terms == "actual":
        data.drop(["Generated Aspect Terms","Generated Sentiment Terms","Generated Polarities","Actual Sentiment Terms"], axis = 1, inplace = True)
    elif aspect_terms == "generated":
        data.drop(["Actual Aspect Terms","Actual Sentiment Terms","Actual Polarities"], axis = 1, inplace = True)
    elif aspect_terms == "none":
        data.drop(["Generated Aspect Terms","Generated Sentiment Terms","Generated Polarities","Actual Aspect Terms","Actual Sentiment Terms","Actual Polarities"], axis = 1, inplace = True)

    if not aspect_polarity:
        try:
            data.drop(["Generated Polarities"],axis=1, inplace = True)
        except:
            pass
        try:
            data.drop(["Actual Polarities"],axis=1, inplace = True)
        except:
            pass

    return data

# Find Aspect Terms

In [ ]:
nlp = stanfordnlp.Pipeline()

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_parser.pt', 'pretrain_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand

In [ ]:
def extract_attributes(text):
    doc = nlp(text)
    parsed_text = {'word':[], 'lemma':[], 'pos':[], 'dep':[], 'gov':[], 'gov_pos':[]}
    for sent in doc.sentences:
        for wrd in sent.words:
            parsed_text['word'].append(wrd.text)
            parsed_text['lemma'].append(wrd.lemma)
            parsed_text['pos'].append(wrd.pos)
            parsed_text['dep'].append(wrd.dependency_relation)
    for sent in doc.sentences:
        for wrd in sent.words:        
            if wrd.governor == 0:
                parsed_text['gov'].append("ROOT")
                parsed_text['gov_pos'].append("-")
            else:   
                parsed_text['gov'].append(parsed_text['word'][wrd.governor-1])
                parsed_text['gov_pos'].append(parsed_text['pos'][wrd.governor-1])
    
    return pd.DataFrame(parsed_text)

In [ ]:
text = "have keep turning until decides lower mute"
extract_attributes(text)

,word,lemma,pos,dep,gov,gov_pos
0,have,have,VBP,aux,turning,VBG
1,keep,keep,NN,nsubj,turning,VBG
2,turning,turn,VBG,root,ROOT,-
3,until,until,IN,mark,decides,VBZ
4,decides,decide,VBZ,advcl,turning,VBG
5,lower,lower,JJR,amod,mute,NN
6,mute,mute,NN,obj,decides,VBZ


In [ ]:
def parse_compound_aspects(data):
    count = 0
    for i in range(len(data)):
        j = 0
        generated_aspect_words_to_be_deleted = set()
        while(j < len(data[i].generated_aspect_words)):
            generated_aspect_term = data[i].generated_aspect_words[j].aspect_term
            individual_aspect_terms = generated_aspect_term.split(" ")
            if(len(individual_aspect_terms) > 1):
                for individual_aspect_term in individual_aspect_terms:
                    k = 0
                    while(k < len(data[i].generated_aspect_words)):
                        if k == j:
                            k += 1
                            continue
                        if individual_aspect_term == data[i].generated_aspect_words[k].aspect_term:
                            data[i].generated_aspect_words[j].sentiment_terms += (data[i].generated_aspect_words[k].sentiment_terms)
                            generated_aspect_words_to_be_deleted.add(data[i].generated_aspect_words[k])
                            count += 1
                        k += 1
            j += 1
        for generated_aspect_word_to_be_deleted in generated_aspect_words_to_be_deleted:
            data[i].generated_aspect_words.remove(generated_aspect_word_to_be_deleted)
    return count

def add_or_update_aspect_word(sentence, aspect_term, sentiment_term):
    if sentiment_term == None:
        sentence.generated_aspect_words.append(AspectWord(aspect_term, []))
        return True
    for i in range(len(sentence.generated_aspect_words)):
        if sentence.generated_aspect_words[i].aspect_term == aspect_term:
            sentence.generated_aspect_words[i].sentiment_terms.append(sentiment_term)
            return True

    sentence.generated_aspect_words.append(AspectWord(aspect_term, [sentiment_term]))
    return True

def generate_aspect_words(data):
        
    for i in range(len(data)):
        extracted_attributes_df = extract_attributes(data[i].sentence)
        data[i].generated_aspect_words.clear()

        # Rules:
        for j in extracted_attributes_df.index:
            
            # Bell, based in Los Angeles, makes and distributes electronic, computer and building products.
            if extracted_attributes_df['dep'][j] == "nsubj":
                # Example: (best, display)
                if extracted_attributes_df['gov_pos'][j] in ["JJ", "JJR", "JJS"] and extracted_attributes_df['pos'][j] in ["NN", "NNS", "NNP"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['word'][j], extracted_attributes_df['gov'][j])
                # Example: (distributes, XYZ Company)
                elif extracted_attributes_df['gov_pos'][j] in ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"] and extracted_attributes_df['pos'][j] in ["NN", "NNS", "NNP"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['word'][j], extracted_attributes_df['gov'][j])
                
            elif extracted_attributes_df['dep'][j] == "amod":
                # Example: (products, electronic)
                if extracted_attributes_df['gov_pos'][j] in ["NN", "NNS", "NNP"] and extracted_attributes_df['pos'][j] in ["JJ", "JJR", "JJS"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['gov'][j], extracted_attributes_df['word'][j])
                # Example: (products, building)
                elif extracted_attributes_df['gov_pos'][j] in ["NN", "NNS", "NNP"] and extracted_attributes_df['pos'][j] in ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['gov'][j], extracted_attributes_df['word'][j])
                
            elif extracted_attributes_df['dep'][j] == "obj":
                # Example: (makes, products)
                if extracted_attributes_df['gov_pos'][j] in ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"] and extracted_attributes_df['pos'][j] in ["NN", "NNS", "NNP"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['word'][j], extracted_attributes_df['gov'][j])
                    
            elif extracted_attributes_df['dep'][j] == "nmod":
                # Both are aspects
                # Example: (display, computer)
                if extracted_attributes_df['gov_pos'][j] in ["NN"] and extracted_attributes_df['pos'][j] in ["NN", "NS"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['word'][j] + " " + extracted_attributes_df['gov'][j], None)
                    
                elif extracted_attributes_df['gov_pos'][j] in ["JJ"] and extracted_attributes_df['pos'][j] in ["NN"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['word'][j], extracted_attributes_df['gov'][j])
                    
            elif extracted_attributes_df['dep'][j] == "acl":
                if extracted_attributes_df['gov_pos'][j] in ["NN"] and extracted_attributes_df['pos'][j] in ["JJ"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['gov'][j], extracted_attributes_df['word'][j])
                
                elif extracted_attributes_df['gov_pos'][j] in ["NNS"] and extracted_attributes_df['pos'][j] in ["VBP"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['gov'][j], extracted_attributes_df['word'][j])
                
            elif extracted_attributes_df['dep'][j] == "conj":
                # Both are aspects
                if extracted_attributes_df['gov_pos'][j] in ["NN"] and extracted_attributes_df['pos'][j] in ["NN", "NNS", "NNP"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['word'][j] + " " + extracted_attributes_df['gov'][j], None)

                elif extracted_attributes_df['gov_pos'][j] in ["NN", "NNS", "NNP"] and extracted_attributes_df['pos'][j] in ["JJ"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['gov'][j], extracted_attributes_df['word'][j])
                    
                elif extracted_attributes_df['gov_pos'][j] in ["NN", "NNS", "NNP"] and extracted_attributes_df['pos'][j] in ["VBZ"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['gov'][j], extracted_attributes_df['word'][j])
                    
            elif extracted_attributes_df['dep'][j] == "compound":
                # Both are aspects
                if extracted_attributes_df['gov_pos'][j] in ["NN"] and extracted_attributes_df['pos'][j] in ["NN"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['word'][j] + " " + extracted_attributes_df['gov'][j], None)
                    
    parse_compound_aspects(data)

# Aspect Term Extraction Performance

In [ ]:
def match_aspect_terms(actual_aspect_term, generated_aspect_term):
    n = len(generated_aspect_term.split(" "))
    for i in range(n):
        temp = generated_aspect_term.split(" ",maxsplit = i)
        if len(temp) > 1:
            rest = temp[1]
        else:
            rest = temp[0]
        if re.search(rest, actual_aspect_term):
            return len(rest)/len(actual_aspect_term)
        generated_aspect_term = rest
    return 0

In [ ]:
def aspect_term_performance(data):
    true_positives = 0
    false_positives = 0
    no_of_actual_aspect_terms = 0
    no_of_generated_aspect_terms = 0

    for i in range(len(data)):
        no_of_generated_aspect_terms += len(data[i].generated_aspect_words)
        no_of_actual_aspect_terms += len(data[i].actual_aspect_words)
        for generated_aspect_word in data[i].generated_aspect_words:
            found = False
            for actual_aspect_word in data[i].actual_aspect_words:
                matched_part = match_aspect_terms(actual_aspect_word.aspect_term, generated_aspect_word.aspect_term)
                if matched_part > 0:
                    true_positives += matched_part
                    found = True
                    break
            if not found:
                false_positives += 1

    ratios = {"Aspect Term Extraction Precision": true_positives / no_of_generated_aspect_terms, "Aspect Term Extraction Recall":true_positives/no_of_actual_aspect_terms, "Aspect Term Extraction True Positives":true_positives, "Actual Aspect Terms":no_of_actual_aspect_terms, "Generated Aspect Terms":no_of_generated_aspect_terms}
    return ratios

# Train the Model
Make a frequency dictionary for all the Sentiment Words

In [ ]:
def train(data):
    # Do preprocessing if needed
    sentiment_terms_frequency = {}
    for sentence in data:
        for generated_aspect_word in sentence.generated_aspect_words:
            polarity = None
            for actual_aspect_word in sentence.actual_aspect_words:
                matched_part = match_aspect_terms(actual_aspect_word.aspect_term, generated_aspect_word.aspect_term)
                if matched_part > 0: 
                    if actual_aspect_word.polarity == 'positive':
                        polarity = 1
                    
                    elif actual_aspect_word.polarity == 'negative':
                        polarity = -1
                    
                    else:
                        polarity = 0
                    break    
            if polarity ==  None:
                continue
            for sentiment_term in generated_aspect_word.sentiment_terms:
                if sentiment_term in list(sentiment_terms_frequency.keys()): 
                    sentiment_terms_frequency[sentiment_term]['frequency'] += 1
                    sentiment_terms_frequency[sentiment_term]['polarity'] += (polarity)
                
                else:
                    sentiment_terms_frequency[sentiment_term] = {'frequency':1, 'polarity':polarity}

    for key in list(sentiment_terms_frequency.keys()):
        sentiment_terms_frequency[key] = (sentiment_terms_frequency[key]['polarity']/sentiment_terms_frequency[key]['frequency'])

    return sentiment_terms_frequency

# Test the Model

In [ ]:
def predict(data, sentiment_terms_frequency):
    generate_aspect_words(data)
    unk_words = set() 
    for i in range(len(data)):
        for j in range(len(data[i].generated_aspect_words)):
            sentiment_terms = data[i].generated_aspect_words[j].sentiment_terms
            unk = 0
            polarity = 0
            for term in sentiment_terms:
                if term not in sentiment_terms_frequency.keys():
                    unk += 1
                    unk_words.add(term)
                else:
                    polarity += sentiment_terms_frequency[term]
            if (len(sentiment_terms) - unk != 0):
                data[i].generated_aspect_words[j].polarity = polarity/(len(sentiment_terms)-unk)
            else:
                data[i].generated_aspect_words[j].polarity = 0
    return len(unk_words)

def find_polarity(value, threshold = 0.2):
    if (value < -1*threshold):
        return "negative"
    elif (value > threshold):
        return "positive"
    else:
        return "neutral"

def classify(data, sentiment_terms_frequency):
    no_of_unk_words = predict(data, sentiment_terms_frequency)
    for i in range(len(data)):
        for j in range(len(data[i].generated_aspect_words)):
            data[i].generated_aspect_words[j].polarity = find_polarity(data[i].generated_aspect_words[j].polarity)
    return no_of_unk_words

# Performance of Sentiment Classification

In [ ]:
# Split data and do this
def sentiment_polarity_classification_performance(data):
    matches = 0
    non_matches = 0
    for i in range(len(data)):
        for actual_aspect_word in data[i].actual_aspect_words:
            actual_aspect_term = actual_aspect_word.aspect_term
            actual_polarity = actual_aspect_word.polarity
            for generated_aspect_word in data[i].generated_aspect_words:
                if (match_aspect_terms(actual_aspect_term, generated_aspect_word.aspect_term) > 0):
                    if (actual_polarity == generated_aspect_word.polarity):
                        matches+=1
                    else:
                        non_matches+=1
                    break

    return {"Sentiment Polarity Matches":matches, "Sentiment Polarity Non-Matches":non_matches, "Sentiment Polarity Classification Precision":matches/(matches+non_matches)}

# Performance of Sentence Polarity Classification

In [ ]:
def sentence_polarity_classification_performance(data): 
    matches = 0
    non_matches = 0
    for i in range(len(data)):
        data[i].calculate_generated_polarity()
        if(data[i].actual_sentence_polarity == data[i].generated_sentence_polarity):
            matches+=1
        else:
            non_matches+=1

    return {"Sentence Polarity Matches":matches, "Sentence Polarity Non Matches":non_matches, "Sentence Polarity Classification Precision":matches/(matches+non_matches)}

# Execution

## Load Data and Generate Aspect Terms

### Laptop

In [ ]:
laptop_train_data = xml_to_sentences(root_dir + train_folder + laptop_train_file)
laptop_train_data_size = len(laptop_train_data)
print(f"Total sentences: {laptop_train_data_size}")
laptop_train_data_df = data_list_to_df(laptop_train_data, aspect_terms = "actual", aspect_polarity = True)
laptop_train_data_df.head()

Total sentences: 1488


,ID,Sentence,Actual Aspect Terms,Actual Polarities
0,1,I charge it at night and skip taking the cord ...,"[cord, battery life]","[neutral, positive]"
1,2,The tech guy then said the service center does...,"[service center, ""sales"" team, tech guy]","[negative, negative, neutral]"
2,3,"it is of high quality, has a killer GUI, is ex...","[quality, GUI, applications, use]","[positive, positive, positive, positive]"
3,4,Easy to start up and does not overheat as much...,[start up],[positive]
4,5,"I even got my teenage son one, because of the ...","[features, iChat, Photobooth, garage band]","[positive, positive, positive, positive]"


In [ ]:
# Split into train and validation
np.random.seed(7)
random.seed(9)
laptop_train_data, laptop_valid_data = sklearn.model_selection.train_test_split(laptop_train_data, test_size=0.1)
print(f"Size of Training Data: {len(laptop_train_data)}\nSize of Validation Data: {len(laptop_valid_data)}")

Size of Training Data: 1339
Size of Validation Data: 149


In [ ]:
generate_aspect_words(laptop_train_data)
laptop_train_data_df = data_list_to_df(laptop_train_data, aspect_terms = "generated", aspect_polarity = True)
laptop_train_data_df.head()

,ID,Sentence,Generated Aspect Terms,Generated Sentiment Terms,Generated Polarities
0,611,Not only are the versions of these programs ab...,[versions],"[[worked, able, superior]]",[None]
1,260,Sometimes you really have to tap the pad to ge...,[pad],[[tap]],[None]
2,799,"-When battery life went to 4 hours or less, to...",[battery life],[[went]],[None]
3,397,If you don't feel comfortable doing it yoursel...,"[case, one, Buy]","[[buy], [white, bought], [Best]]","[None, None, None]"
4,834,/ awesome cooling system/ much better grafics ...,"[cooling system, card, GB RAM, RAM screen, bac...","[[awesome], [better], [], [LED], [LED], [LED, ...","[None, None, None, None, None, None]"


In [ ]:
laptop_train_data_ratios = aspect_term_performance(laptop_train_data)
pd.DataFrame.from_dict(laptop_train_data_ratios, orient = 'index',columns = ["Parameters"])

,Parameters
Aspect Term Extraction Precision,0.419667
Aspect Term Extraction Recall,0.566810
Aspect Term Extraction True Positives,1191.433998
Actual Aspect Terms,2102.000000
Generated Aspect Terms,2839.000000


### Restaurant

In [ ]:
restaurant_train_data = xml_to_sentences(root_dir + train_folder + restaurant_train_file)
restaurant_train_data_size = len(restaurant_train_data)
print(f"Total sentences: {restaurant_train_data_size}")
restaurant_train_data_df = data_list_to_df(restaurant_train_data, aspect_terms = "actual", aspect_polarity = True)
restaurant_train_data_df.head()

Total sentences: 2021


,ID,Sentence,Actual Aspect Terms,Actual Polarities
0,1,But the staff was so horrible to us.,[staff],[negative]
1,2,"To be completely fair, the only redeeming fact...",[food],[positive]
2,3,"The food is uniformly exceptional, with a very...","[food, kitchen, menu]","[positive, positive, neutral]"
3,4,"Not only was the food outstanding, but the lit...","[food, perks]","[positive, positive]"
4,5,Our agreed favorite is the orrechiete with sau...,"[orrechiete with sausage and chicken, waiters,...","[positive, positive, neutral, neutral]"


In [ ]:
# Split into train and validation
np.random.seed(7)
random.seed(9)
restaurant_train_data, restaurant_valid_data = sklearn.model_selection.train_test_split(restaurant_train_data, test_size=0.1)
print(f"Size of Training Data: {len(restaurant_train_data)}\nSize of Validation Data: {len(restaurant_valid_data)}")

Size of Training Data: 1818
Size of Validation Data: 203


In [ ]:
generate_aspect_words(restaurant_train_data)
restaurant_train_data_df = data_list_to_df(restaurant_train_data, aspect_terms = "generated", aspect_polarity = True)
restaurant_train_data_df.head()

,ID,Sentence,Generated Aspect Terms,Generated Sentiment Terms,Generated Polarities
0,1706,I had the mango chicken and i can't go on to t...,"[mango chicken, presentation]","[[had], [beatiful]]","[None, None]"
1,1987,"I've had the chicken with garlic sauce, chicke...","[garlic sauce, sauce chicken, chicken sauce, b...","[[black], [black, had], [had, black], [black],...","[None, None, None, None, None, None, None]"
2,652,My friend got the mushroom pizza which tasted ...,"[friend, mushroom pizza]","[[got], [got]]","[None, None]"
3,142,What came to our table was burned beyond recog...,[recognition],[[stringy]],[None]
4,354,"otherwise, good stuff for late nite eats.","[stuff, eats]","[[good], [late]]","[None, None]"


In [ ]:
restaurant_train_data_ratios = aspect_term_performance(restaurant_train_data)
pd.DataFrame.from_dict(restaurant_train_data_ratios, orient = 'index',columns = ["Parameters"])

,Parameters
Aspect Term Extraction Precision,0.571080
Aspect Term Extraction Recall,0.642768
Aspect Term Extraction True Positives,2114.707670
Actual Aspect Terms,3290.000000
Generated Aspect Terms,3703.000000


## Training

### Laptop

In [ ]:
laptop_sentiment_terms_frequency_train = train(laptop_train_data)
print(f"Number of Sentiment Terms: {len(laptop_sentiment_terms_frequency_train)}")
df = pd.DataFrame.from_dict(laptop_sentiment_terms_frequency_train, orient = 'index', columns =["Polarity"])
df.head()

Number of Sentiment Terms: 615


,Polarity
tap,-1.000000
went,-0.833333
buy,-0.333333
awesome,1.000000
better,0.142857


### Restaurant

In [ ]:
restaurant_sentiment_terms_frequency_train = train(restaurant_train_data)
print(f"Number of Sentiment Terms: {len(restaurant_sentiment_terms_frequency_train)}")
df = pd.DataFrame.from_dict(restaurant_sentiment_terms_frequency_train, orient = 'index', columns =["Polarity"])
df.head()

Number of Sentiment Terms: 853


,Polarity
had,0.406977
beatiful,1.000000
black,0.363636
got,0.714286
good,0.816327


### Mixed

In [ ]:
mixed_sentiment_terms_frequency_train = train(laptop_train_data + restaurant_train_data)
print(f"Number of Sentiment Terms: {len(mixed_sentiment_terms_frequency_train)}")
df = pd.DataFrame.from_dict(mixed_sentiment_terms_frequency_train, orient = 'index', columns =["Polarity"])
df.head()

Number of Sentiment Terms: 1251


,Polarity
tap,-1.000000
went,-0.833333
buy,-0.272727
awesome,1.000000
better,0.166667


## Validation

### Laptop

In [ ]:
laptop_valid_data_predictions_unk_count = classify(laptop_valid_data, mixed_sentiment_terms_frequency_train)
# laptop_valid_data_predictions_unk_count = classify(laptop_valid_data, laptop_sentiment_terms_frequency_train)
print("Unknown Sentiment Terms:", laptop_valid_data_predictions_unk_count)
laptop_valid_data_df = data_list_to_df(laptop_valid_data, aspect_terms = "none", aspect_polarity = False)
laptop_valid_data_df.head()

Unknown Sentiment Terms: 66


,ID,Sentence
0,433,"WHEN TYPING, LETTERS AND SPACES ARE FREQUENTLY..."
1,59,I love the glass touchpad.
2,1021,Later it held zero charge and its replacemen...
3,678,This computer is exceptionally thin for it's s...
4,1319,It has just enough RAM to run smoothly and eno...


In [ ]:
laptop_valid_data_ratios = aspect_term_performance(laptop_valid_data)
laptop_valid_data_ratios.update(sentiment_polarity_classification_performance(laptop_valid_data))
laptop_valid_data_ratios.update(sentence_polarity_classification_performance(laptop_valid_data))
pd.DataFrame.from_dict(laptop_valid_data_ratios, orient = 'index',columns = ["Parameters"])

,Parameters
Aspect Term Extraction Precision,0.467085
Aspect Term Extraction Recall,0.607576
Aspect Term Extraction True Positives,155.539348
Actual Aspect Terms,256.000000
Generated Aspect Terms,333.000000
Sentiment Polarity Matches,91.000000
Sentiment Polarity Non-Matches,78.000000
Sentiment Polarity Classification Precision,0.538462
Sentence Polarity Matches,65.000000
Sentence Polarity Non Matches,84.000000


### Restaurant

In [ ]:
restaurant_valid_data_predictions_unk_count = classify(restaurant_valid_data, mixed_sentiment_terms_frequency_train)
# restaurant_valid_data_predictions_unk_count = classify(restaurant_valid_data, restaurant_sentiment_terms_frequency_train)
print("Unknown Sentiment Terms:", restaurant_valid_data_predictions_unk_count)
restaurant_valid_data_df = data_list_to_df(restaurant_valid_data, aspect_terms = "none", aspect_polarity = False)
restaurant_valid_data_df.head()

Unknown Sentiment Terms: 78


,ID,Sentence
0,1396,"Their sushi, Kamikaze and other Rolls are fres..."
1,141,I had very high expectations for this place an...
2,111,"Pizza and garlic knots are great as well, I or..."
3,638,No free drink.
4,702,I tend to judge a sushi restaurant by its sea ...


In [ ]:
restaurant_valid_data_ratios = aspect_term_performance(restaurant_valid_data)
restaurant_valid_data_ratios.update(sentiment_polarity_classification_performance(restaurant_valid_data))
restaurant_valid_data_ratios.update(sentence_polarity_classification_performance(restaurant_valid_data))
pd.DataFrame.from_dict(restaurant_valid_data_ratios, orient = 'index',columns = ["Parameters"])

,Parameters
Aspect Term Extraction Precision,0.601044
Aspect Term Extraction Recall,0.675615
Aspect Term Extraction True Positives,272.272819
Actual Aspect Terms,403.000000
Generated Aspect Terms,453.000000
Sentiment Polarity Matches,161.000000
Sentiment Polarity Non-Matches,120.000000
Sentiment Polarity Classification Precision,0.572954
Sentence Polarity Matches,123.000000
Sentence Polarity Non Matches,80.000000


## Testing

### Laptop

#### Test-1 Data

In [ ]:
laptop_test_1_data = xml_to_sentences(root_dir + test_1_folder + laptop_test_1_file, data_type = "Test")
laptop_test_1_data_df = data_list_to_df(laptop_test_1_data, aspect_terms = "none", aspect_polarity = False)
laptop_test_1_data_df.head()

,ID,Sentence
0,1,"Boot time is super fast, around anywhere from ..."
1,2,tech support would not fix the problem unless ...
2,3,but in resume this computer rocks!
3,4,Set up was easy.
4,5,Did not enjoy the new Windows 8 and touchscree...


In [ ]:
laptop_test_1_predictions_unk_count = classify(laptop_test_1_data, mixed_sentiment_terms_frequency_train)
print("Number of Unknown Sentiment Terms:", laptop_test_1_predictions_unk_count)
laptop_test_1_data_df = data_list_to_df(laptop_test_1_data, aspect_terms = "generated", aspect_polarity = True)
laptop_test_1_data_df.head()

Number of Unknown Sentiment Terms: 202


,ID,Sentence,Generated Aspect Terms,Generated Sentiment Terms,Generated Polarities
0,1,"Boot time is super fast, around anywhere from ...",[Boot time],[[fast]],[positive]
1,2,tech support would not fix the problem unless ...,"[tech support, problem, plan]","[[fix], [fix], [bought]]","[negative, negative, negative]"
2,3,but in resume this computer rocks!,[computer],[[rocks]],[neutral]
3,4,Set up was easy.,[],[],[]
4,5,Did not enjoy the new Windows 8 and touchscree...,"[functions, Windows]","[[new, enjoy], [enjoy]]","[positive, positive]"


#### Test-2 Data

In [ ]:
laptop_test_2_data = xml_to_sentences(root_dir + test_2_folder + laptop_test_2_file, data_type = "Test-2")
laptop_test_2_data_df = data_list_to_df(laptop_test_2_data, aspect_terms = "none", aspect_polarity = False)
laptop_test_2_data_df.head()

,ID,Sentence
0,1,"Boot time is super fast, around anywhere from ..."
1,2,tech support would not fix the problem unless ...
2,3,Set up was easy.
3,4,Did not enjoy the new Windows 8 and touchscree...
4,5,Other than not being a fan of click pads (indu...


In [ ]:
laptop_test_2_predictions_unk_count = classify(laptop_test_2_data, mixed_sentiment_terms_frequency_train)
print("Number of Unknown Sentiment Terms:", laptop_test_2_predictions_unk_count)
laptop_test_2_data_df = data_list_to_df(laptop_test_2_data, aspect_terms = "both", aspect_polarity = True)
laptop_test_2_data_df.head()

Number of Unknown Sentiment Terms: 127


,ID,Sentence,Actual Aspect Terms,Actual Sentiment Terms,Actual Polarities,Generated Aspect Terms,Generated Sentiment Terms,Generated Polarities
0,1,"Boot time is super fast, around anywhere from ...",[Boot time],[[]],[None],[Boot time],[[fast]],[positive]
1,2,tech support would not fix the problem unless ...,[tech support],[[]],[None],"[tech support, problem, plan]","[[fix], [fix], [bought]]","[negative, negative, negative]"
2,3,Set up was easy.,[Set up],[[]],[None],[],[],[]
3,4,Did not enjoy the new Windows 8 and touchscree...,"[Windows 8, touchscreen functions]","[[], []]","[None, None]","[functions, Windows]","[[new, enjoy], [enjoy]]","[positive, positive]"
4,5,Other than not being a fan of click pads (indu...,"[internal speakers, price tag, click pads]","[[], [], []]","[None, None, None]","[industry standard, speakers, things, price tag]","[[], [lousy, internal], [find], [considering]]","[neutral, neutral, neutral, positive]"


In [ ]:
laptop_test_2_data_ratios = aspect_term_performance(laptop_test_2_data)
pd.DataFrame.from_dict(laptop_test_2_data_ratios, orient = 'index',columns = ["Parameters"])

,Parameters
Aspect Term Extraction Precision,0.464897
Aspect Term Extraction Recall,0.545224
Aspect Term Extraction True Positives,356.576322
Actual Aspect Terms,654.000000
Generated Aspect Terms,767.000000


### Restaurant

#### Test-1 Data

In [ ]:
restaurant_test_1_data = xml_to_sentences(root_dir + test_1_folder + restaurant_test_1_file, data_type = "Test")
restaurant_test_1_data_df = data_list_to_df(restaurant_test_1_data, aspect_terms = "none", aspect_polarity = False)
restaurant_test_1_data_df.head()

,ID,Sentence
0,1,The bread is top notch as well.
1,2,I have to say they have one of the fastest del...
2,3,Food is always fresh and hot- ready to eat!
3,4,Did I mention that the coffee is OUTSTANDING?
4,5,"Certainly not the best sushi in New York, howe..."


In [ ]:
restaurant_test_1_predictions_unk_count = classify(restaurant_test_1_data, mixed_sentiment_terms_frequency_train)
print("Number of Unknown Sentiment Terms:", restaurant_test_1_predictions_unk_count)
restaurant_test_1_data_df = data_list_to_df(restaurant_test_1_data, aspect_terms = "generated", aspect_polarity = True)
restaurant_test_1_data_df.head()

Number of Unknown Sentiment Terms: 278


,ID,Sentence,Generated Aspect Terms,Generated Sentiment Terms,Generated Polarities
0,1,The bread is top notch as well.,[notch],[[top]],[positive]
1,2,I have to say they have one of the fastest del...,[times],[[fastest]],[positive]
2,3,Food is always fresh and hot- ready to eat!,[Food],[[fresh]],[positive]
3,4,Did I mention that the coffee is OUTSTANDING?,[coffee],[[OUTSTANDING]],[neutral]
4,5,"Certainly not the best sushi in New York, howe...","[sushi, place]","[[best], [clean]]","[positive, positive]"


#### Test-2 Data

In [ ]:
restaurant_test_2_data = xml_to_sentences(root_dir + test_2_folder + restaurant_test_2_file, data_type = "Test-2")
restaurant_test_2_data_df = data_list_to_df(restaurant_test_2_data, aspect_terms = "none", aspect_polarity = False)
restaurant_test_2_data_df.head()

,ID,Sentence
0,1,The bread is top notch as well.
1,2,I have to say they have one of the fastest del...
2,3,Food is always fresh and hot- ready to eat!
3,4,Did I mention that the coffee is OUTSTANDING?
4,5,"Certainly not the best sushi in New York, howe..."


In [ ]:
restaurant_test_2_predictions_unk_count = classify(restaurant_test_2_data, mixed_sentiment_terms_frequency_train)
print("Number of Unknown Sentiment Terms:", restaurant_test_2_predictions_unk_count)
restaurant_test_2_data_df = data_list_to_df(restaurant_test_2_data, aspect_terms = "both", aspect_polarity = True)
restaurant_test_2_data_df.head()

Number of Unknown Sentiment Terms: 239


,ID,Sentence,Actual Aspect Terms,Actual Sentiment Terms,Actual Polarities,Generated Aspect Terms,Generated Sentiment Terms,Generated Polarities
0,1,The bread is top notch as well.,[bread],[[]],[None],[notch],[[top]],[positive]
1,2,I have to say they have one of the fastest del...,[delivery times],[[]],[None],[times],[[fastest]],[positive]
2,3,Food is always fresh and hot- ready to eat!,[Food],[[]],[None],[Food],[[fresh]],[positive]
3,4,Did I mention that the coffee is OUTSTANDING?,[coffee],[[]],[None],[coffee],[[OUTSTANDING]],[neutral]
4,5,"Certainly not the best sushi in New York, howe...","[sushi, place]","[[], []]","[None, None]","[sushi, place]","[[best], [clean]]","[positive, positive]"


In [ ]:
restaurant_test_2_data_ratios = aspect_term_performance(restaurant_test_2_data)
pd.DataFrame.from_dict(restaurant_test_2_data_ratios, orient = 'index',columns = ["Parameters"])

,Parameters
Aspect Term Extraction Precision,0.599928
Aspect Term Extraction Recall,0.643309
Aspect Term Extraction True Positives,729.512554
Actual Aspect Terms,1134.000000
Generated Aspect Terms,1216.000000


## Save to CSV

In [ ]:
def write_to_csv(data, path):
    data_df = data_list_to_df(data, aspect_terms = "both", aspect_polarity = True, no_of_rows = len(data))
    data_df.to_csv(path_or_buf = path)

write_to_csv(laptop_train_data, root_dir + train_folder + "generated_laptop_train_data.csv")
write_to_csv(laptop_valid_data, root_dir + train_folder + "generated_laptop_valid_data.csv")
write_to_csv(laptop_test_1_data, root_dir + test_1_folder + "generated_laptop_test_1_data.csv")
write_to_csv(laptop_test_2_data, root_dir + test_2_folder + "generated_laptop_test_2_data.csv")
write_to_csv(restaurant_train_data, root_dir + train_folder + "generated_restaurant_train_data.csv")
write_to_csv(restaurant_valid_data, root_dir + train_folder + "generated_restaurant_valid_data.csv")
write_to_csv(restaurant_test_1_data, root_dir + test_1_folder + "generated_restaurant_test_1_data.csv")
write_to_csv(restaurant_test_2_data, root_dir + test_2_folder + "generated_restaurant_test_2_data.csv")

# Demo


In [ ]:
def test_classify(string,sentiment_terms_frequency_train):
    test_data = []
    test_data.append(Sentence(1,string,"Test"))
    test_predictions_unk_count = classify(test_data, mixed_sentiment_terms_frequency_train)
    print(f"Unknown Sentiment Words: {test_predictions_unk_count}\n\nAnalysis:")
    test_data_df = data_list_to_df(test_data, aspect_terms = "generated", aspect_polarity = True)
    return (test_data_df)

Positive Polarity

In [ ]:
test_classify("Also the display is exceptional!",mixed_sentiment_terms_frequency_train)

Unknown Sentiment Words: 0

Analysis:


,ID,Sentence,Generated Aspect Terms,Generated Sentiment Terms,Generated Polarities
0,1,Also the display is exceptional!,[display],[[exceptional]],[positive]


Negative Polarity

In [ ]:
test_classify("The display is beyond horrible.",mixed_sentiment_terms_frequency_train)

Unknown Sentiment Words: 0

Analysis:


,ID,Sentence,Generated Aspect Terms,Generated Sentiment Terms,Generated Polarities
0,1,The display is beyond horrible.,[display],[[horrible]],[negative]


Multiple Aspect Terms

In [ ]:
test_classify("Screen is awesome, battery life is good.",mixed_sentiment_terms_frequency_train)

Unknown Sentiment Words: 0

Analysis:


,ID,Sentence,Generated Aspect Terms,Generated Sentiment Terms,Generated Polarities
0,1,"Screen is awesome, battery life is good.","[Screen, battery life]","[[awesome], [good]]","[positive, positive]"


Multiple Sentiment Words

In [ ]:
test_classify("Its white color is stylish for college students and easy to take to carry and take to classes.",mixed_sentiment_terms_frequency_train)

Unknown Sentiment Words: 0

Analysis:


,ID,Sentence,Generated Aspect Terms,Generated Sentiment Terms,Generated Polarities
0,1,Its white color is stylish for college student...,[color],"[[white, stylish]]",[positive]


Compound Aspect Words

In [ ]:
test_classify("It's also fairly easy to use the Operating System.",mixed_sentiment_terms_frequency_train)

Unknown Sentiment Words: 0

Analysis:


,ID,Sentence,Generated Aspect Terms,Generated Sentiment Terms,Generated Polarities
0,1,It's also fairly easy to use the Operating Sys...,[Operating System],[[use]],[positive]


# Shortcomings and improvements

* Some compound aspect terms lack corresponding sentiment words, because despite the abundant rules, a few cases could not be accounted for. For these cases we can use N-gram model to have a prediction, even if it is worse.
* Adjectival Modifiers (like "very good") are not taken into consideration, as adding those rules fragments the polarity of the base sentiment ("good") which causes a drpo in precision.
* Negation should be handled in preprocessing and the sentiment converted to the opposite there only. This is not possible without a pretrained model which can find antonyms.

